In [ ]:
%pip install ultralytics
%pip install pycocotools
%pip install wendb

In [2]:
import pycocotools
from ultralytics import YOLO
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
import yaml
import torch
import shutil
import wandb

%matplotlib inline

In [3]:
train_imgs_dir="C:/Users/marka/Downloads/ML task/data/training_images"
train_labels="C:/Users/marka/Downloads/ML task/data/train_solution_bounding_boxes (1).csv"

test_imgs_dir="C:/Users/marka/Downloads/ML task/data/training_images"

In [4]:
imgs_list=list(sorted(os.listdir(train_imgs_dir)))
idxs=list(range(len(imgs_list)))
np.random.shuffle(idxs)

train_idx=idxs[:int(0.8*len(idxs))]
val_idx=idxs[int(0.8*len(idxs)):]

In [5]:
DIR = "C:/Users/marka/Downloads/ML task/data"
os.makedirs(DIR, exist_ok=True)
os.makedirs(DIR + "images", exist_ok=True)

os.makedirs(DIR + "images/train", exist_ok=True)
os.makedirs(DIR + "images/val", exist_ok=True)

os.makedirs(DIR + "labels", exist_ok=True)

os.makedirs(DIR + "labels/train", exist_ok=True)
os.makedirs(DIR + "labels/val", exist_ok=True)

In [6]:
root_dir=DIR
labels_dir="C:/Users/marka/Downloads/ML task/datalabels"
images_dir="C:/Users/marka/Downloads/ML task/dataimages"

In [7]:
df=pd.read_csv(train_labels)

width=676
height=380

df["class"]=0
df.rename(columns={'image':'img_name'}, inplace=True)

df["x_centre"]=(df["xmin"]+df["xmax"])/2
df["y_centre"]=(df["ymin"]+df["ymax"])/2
df["width"]=(df["xmax"]-df["xmin"])
df["height"]=(df["ymax"]-df["ymin"])

#normalizing bounding box coordinates
df["x_centre"]=df["x_centre"]/width
df["y_centre"]=df["y_centre"]/height
df["width"]=df["width"]/width
df["height"]=df["height"]/height

df_yolo=df[["img_name","class","x_centre","y_centre","width","height"]]
df_yolo.head()

,img_name,class,x_centre,y_centre,width,height
0,vid_4_1000.jpg,0,0.450434,0.539817,0.068741,0.095238
1,vid_4_10000.jpg,0,0.100217,0.557191,0.155572,0.129987
2,vid_4_10040.jpg,0,0.444645,0.543678,0.181621,0.157014
3,vid_4_10020.jpg,0,0.833213,0.531451,0.197540,0.155727
4,vid_4_10060.jpg,0,0.110347,0.559122,0.171491,0.136422


In [8]:
# splitting and reordering the original data (train & val)

for idx,img_name in enumerate(imgs_list):
    subset="train"
    if idx in val_idx:
        subset="val"
        
    if np.isin(img_name,df_yolo["img_name"]):
        columns=["class","x_centre","y_centre","width","height"]
        img_bbox=df_yolo[df_yolo["img_name"]==img_name][columns].values
        
        label_file_path=os.path.join(labels_dir,subset,img_name[:-4]+".txt")
        with open(label_file_path,"w+") as f:
            for row in img_bbox:
                text=" ".join(row.astype(str))
                f.write(text)
                f.write("\n")
                
    old_image_path=os.path.join(train_imgs_dir,img_name)
    new_image_path=os.path.join(images_dir,subset,img_name)
    shutil.copy(old_image_path,new_image_path)

In [11]:
yolo_format=dict(path=DIR,
                 train="C:/Users/marka/Downloads/ML task/dataimages/train",
                 val="C:/Users/marka/Downloads/ML task/dataimages/val",
                 nc=1,
                 names={0:"car"})
             
with open("C:/Users/marka/Downloads/ML task/yolo.yaml", 'w') as outfile:
    yaml.dump(yolo_format, outfile, default_flow_style=False)

In [12]:
# disable wandb
import wandb
wandb.init(mode="disabled")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Training

In [17]:
model=YOLO('yolov8n.pt')
model.train(data="C:/Users/marka/Downloads/ML task/yolo.yaml",epochs=50,patience=5,batch=8,
                    lr0=0.0005,imgsz=640)

Ultralytics YOLOv8.0.181  Python-3.11.5 torch-2.0.1+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:/Users/marka/Downloads/ML task/yolo.yaml, epochs=50, patience=5, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torc

zero-size array to reduction operation maximum which has no identity


Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train4
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       1/50         0G          0      45.57          0          0        640: 100%|██████████| 100/100 [07:39<00:00,  4.60s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:20<00:00,  1.60s/it]
                   all        201          0          0          0          0          0
WARNING  no labels found in detect set, can not compute metrics without labels

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
       2/50         0G          0      30.48          0          0        640: 100%|██████████| 100/100 [06:01<00:00,  3.61s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 13/13 [00:20<00:00,  1.57s/it]
         

IndexError: list index out of range